In [1]:
# Import dependencies
import pandas as pd
import gmaps
import requests
import numpy as np
import gmaps.datasets

# Import the API key.
from config import g_key

# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [2]:
# Import CSV files as a new dataframe
vacation_data_df = pd.read_csv("weather_data/WeatherPy_vacation.csv", index_col = 0)

In [3]:
vacation_data_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
City_ID,,,,,,,
3,Virginia Beach,US,82.00,scattered clouds,36.85,-75.98,Hilton Virginia Beach Oceanfront
6,Mahebourg,MU,71.60,light intensity shower rain,-20.41,57.70,Shandrani Beachcomber Resort & Spa
7,Atuona,PF,79.84,clear sky,-9.80,-139.03,Villa Enata
9,Jalu,LY,76.44,clear sky,29.03,21.55,Jalu Hotel
10,Rikitea,PF,72.36,overcast clouds,-23.12,-134.97,Pension Maro'i
13,Kiunga,PG,83.86,broken clouds,-6.12,141.29,Airport Lodge Kiunga
14,Sovetskiy,RU,73.44,overcast clouds,61.36,63.58,Apartment on Kievskaya 29
18,Butaritari,KI,78.64,overcast clouds,3.07,172.79,Isles Sunset Lodge
20,Ponta Do Sol,PT,71.01,clear sky,32.67,-17.10,Hotel do Campo


In [4]:
vacation_data_df.index.name = None
vacation_data_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Virginia Beach,US,82.00,scattered clouds,36.85,-75.98,Hilton Virginia Beach Oceanfront
6,Mahebourg,MU,71.60,light intensity shower rain,-20.41,57.70,Shandrani Beachcomber Resort & Spa
7,Atuona,PF,79.84,clear sky,-9.80,-139.03,Villa Enata
9,Jalu,LY,76.44,clear sky,29.03,21.55,Jalu Hotel
10,Rikitea,PF,72.36,overcast clouds,-23.12,-134.97,Pension Maro'i
13,Kiunga,PG,83.86,broken clouds,-6.12,141.29,Airport Lodge Kiunga
14,Sovetskiy,RU,73.44,overcast clouds,61.36,63.58,Apartment on Kievskaya 29
18,Butaritari,KI,78.64,overcast clouds,3.07,172.79,Isles Sunset Lodge
20,Ponta Do Sol,PT,71.01,clear sky,32.67,-17.10,Hotel do Campo
26,Hobyo,SO,78.64,overcast clouds,5.35,48.53,Osmani Hotel


In [5]:
# Convert Max Temp from float to string.
vacation_data_df['Max Temp'] = vacation_data_df['Max Temp'].astype(str)

In [6]:
# Creating a new column with Current Weather by merging Current Description and Max Temp into one.
vacation_data_df["Current Weather"] = vacation_data_df["Current Description"] +" and "+ vacation_data_df["Max Temp"] +" " u"\N{DEGREE SIGN}""F"

# Dropping the columns for Current Description and Max Temp as the information has been replaced with Current Weather
vacation_data_df = vacation_data_df.drop(['Current Description', 'Max Temp'], axis=1)

# Rearranging the columns in DataFrame
vacation_data_df = vacation_data_df[["City", "Country", "Lat", "Lng", "Current Weather", "Hotel Name"]]

In [7]:
vacation_data_df.head(10)

,City,Country,Lat,Lng,Current Weather,Hotel Name
3,Virginia Beach,US,36.85,-75.98,scattered clouds and 82.0 °F,Hilton Virginia Beach Oceanfront
6,Mahebourg,MU,-20.41,57.70,light intensity shower rain and 71.6 °F,Shandrani Beachcomber Resort & Spa
7,Atuona,PF,-9.80,-139.03,clear sky and 79.84 °F,Villa Enata
9,Jalu,LY,29.03,21.55,clear sky and 76.44 °F,Jalu Hotel
10,Rikitea,PF,-23.12,-134.97,overcast clouds and 72.36 °F,Pension Maro'i
13,Kiunga,PG,-6.12,141.29,broken clouds and 83.86 °F,Airport Lodge Kiunga
14,Sovetskiy,RU,61.36,63.58,overcast clouds and 73.44 °F,Apartment on Kievskaya 29
18,Butaritari,KI,3.07,172.79,overcast clouds and 78.64 °F,Isles Sunset Lodge
20,Ponta Do Sol,PT,32.67,-17.10,clear sky and 71.01 °F,Hotel do Campo
26,Hobyo,SO,5.35,48.53,overcast clouds and 78.64 °F,Osmani Hotel


In [8]:
# Creating a pop up marker for the DataFrame to allow layering on a map.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Weather}</dd>
</dl>
"""

In [9]:
# Store the DataFrame Row.
vacation_info = [info_box_template.format(**row) for index, row in vacation_data_df.iterrows()]

In [10]:
# map with selected vacation spots.
locations = vacation_data_df[["Lat", "Lng"]]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=vacation_info)

fig.add_layer(marker_layer)

# Call fig data.
fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
# Choose four cities, create a dataframe and use loc attribute
buchanan_df = vacation_data_df.loc[(vacation_data_df["City"] == "Buchanan")]
grand_lahou_df = vacation_data_df.loc[(vacation_data_df["City"] == "Grand-Lahou")]
takoradi_df = vacation_data_df.loc[(vacation_data_df["City"] == "Takoradi")]
mumford_df = vacation_data_df.loc[(vacation_data_df["City"] == "Mumford")]

In [12]:
takoradi_df

,City,Country,Lat,Lng,Current Weather,Hotel Name
551,Takoradi,GH,4.88,-1.76,scattered clouds and 73.33 °F,Raybow International Hotel


In [13]:
# List of cities
buchanan = (np.array(buchanan_df)[0,2], np.array(buchanan_df)[0,3])
grand_lahou = (np.array(grand_lahou_df)[0,2], np.array(grand_lahou_df)[0,3])
takoradi = (np.array(takoradi_df)[0,2], np.array(takoradi_df)[0,3])
mumford = (np.array(mumford_df)[0,2], np.array(mumford_df)[0,3])

In [14]:
# test = gmaps.directions_layer(buchanan, mumford)
# fig.add_layer(test)
# fig

In [15]:
# Create dataframe of vacation cities
city_marker_df = pd.concat([buchanan_df, grand_lahou_df, takoradi_df, mumford_df])

In [16]:
city_marker_df.head()

,City,Country,Lat,Lng,Current Weather,Hotel Name
275,Buchanan,LR,5.88,-10.05,overcast clouds and 74.53 °F,Kamaneahn Hotel Inc
75,Grand-Lahou,CI,5.14,-5.02,broken clouds and 73.99 °F,Hotel Beatitude de Grand Lahou
551,Takoradi,GH,4.88,-1.76,scattered clouds and 73.33 °F,Raybow International Hotel
326,Mumford,GH,5.26,-0.76,clear sky and 73.6 °F,Spanish Villa


In [21]:
# Driving travel mode 
fig = gmaps.figure()

# Travel Marker -**below code did not work** - after enabling directions API, it says I am not allowed to work with google maps. LA Also tried to help but no luck. 
#african_cities_trip = gmaps.directions_layer(buchanan, mumford, waypoints=[grand_lahou, takoradi], 
#    travel_mode ='DRIVING')
african_cities_trip = gmaps.directions_layer(buchanan, mumford)
fig.add_layer(african_cities_trip)
fig

Figure(layout=FigureLayout(height='420px'))

In [22]:
# Create pop up marker for the DataFrame for layering on a map.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Weather}</dd>
</dl>
"""

In [23]:
# Store the dataframe row
vacation_info = [info_box_template.format(**row) for index, row in city_marker_df.iterrows()]

In [24]:
# map with selected vacation spots.
locations = city_marker_df[["Lat", "Lng"]]

fig = gmaps.figure(center=(5.0, -4.0), zoom_level=6.2)

marker_layer = gmaps.marker_layer(locations, info_box_content=vacation_info)

fig.add_layer(marker_layer)

# Call fig data.
fig

Figure(layout=FigureLayout(height='420px'))